### Imports

In [1]:
import os
import numpy as np
import time
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader
from chess import pgn
from tqdm import tqdm


### Loading Data

In [2]:

def load_pgn(file_path):
    games = []
    with open(file_path, 'r', encoding='utf-8', errors='ignore') as pgn_file:
        while True:
            game = pgn.read_game(pgn_file)
            if game is None:
                break
            games.append(game)
    return games

files = [file for file in os.listdir("/home/rusted/Projects/chessengine/data/pgn") if file.endswith(".pgn")]
#FILE_LIMIT = min(len(files),28)
games = []
i = 1
for file in tqdm(files):
    games.extend(load_pgn(f"/home/rusted/Projects/chessengine/data/pgn/{file}"))
    #if i >= FILE_LIMIT:
    break
    i += 1 

  0%|          | 0/1 [00:17<?, ?it/s]


In [3]:
len(games)

8439

### Convert data to tensors

In [4]:
from auxiliary_functions import create_input_for_nn, encode_moves

In [5]:
X, y = create_input_for_nn(games)

print(f"SAMPLES : {len(y)}")

SAMPLES : 723923


In [6]:
X = X[0:1000000]
y = y[0:1000000]

In [7]:
y, move_to_int = encode_moves(y)
num_classes = len(move_to_int)

In [8]:
X = torch.tensor(X, dtype=torch.float32)
y = torch.tensor(y, dtype=torch.long)

### Training

In [9]:
from dataset import ChessDataset
from model import ChessModel

In [10]:
# Make dataset and DataLoader
dataset = ChessDataset(X, y)
dataloader = DataLoader(dataset, batch_size=64, shuffle=True)

# CUDA
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f'Using device: {device}')

# Initializing the model
model = ChessModel(num_classes=num_classes).to(device)
loss_criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=0.0001)


Using device: cuda


In [11]:
num_epochs = 50
for epoch in range(num_epochs):
    start_time = time.time()
    model.train()
    total_loss = 0.0
    for inputs, labels in tqdm(dataloader):
        inputs, labels = inputs.to(device), labels.to(device) # Move data to GPU
        optimizer.zero_grad()
        
        outputs = model(inputs) # Logits

        # Compute loss
        loss = loss_criterion(outputs, labels)
        loss.backward()

        # Gradient clipping
        torch.nn.utils.clip_grad_norm_(model.parameters(), max_norm=1.0)

        optimizer.step()
        total_loss += loss.item()
    end_time = time.time()
    epoch_time = end_time - start_time
    print(f'Epoch {epoch + 1}/{num_epochs}, Loss: {total_loss / len(dataloader):.4f}, Time: {epoch_time}')

100%|██████████| 11312/11312 [00:37<00:00, 301.58it/s]


Epoch 1/50, Loss: 4.5913, Time: 37.51087141036987


100%|██████████| 11312/11312 [00:37<00:00, 304.89it/s]


Epoch 2/50, Loss: 3.4590, Time: 37.10342717170715


100%|██████████| 11312/11312 [00:39<00:00, 284.44it/s]


...

Epoch 48/50, Loss: 0.7969, Time: 32.33605241775513


100%|██████████| 11312/11312 [00:37<00:00, 300.86it/s]


Epoch 49/50, Loss: 0.7795, Time: 37.600913524627686


100%|██████████| 11312/11312 [00:35<00:00, 314.32it/s]

Epoch 50/50, Loss: 0.7647, Time: 35.99007201194763


### Save model and mappings

In [12]:
torch.save(model.state_dict(),"/home/rusted/Projects/chessengine/models/CHESSDATA2.pth")

In [13]:
import pickle
with open("/home/rusted/Projects/chessengine/models/move_to_int2", "wb") as file:
    pickle.dump(move_to_int, file)